# Setting up Y_Bus, bus and line objects for all task

In [1]:
from classes import *
from functions import *

# Choose data
line_data = ReadCsvFile('./files/given_network/network_configuration_line_data.csv')
# line_data = ReadCsvFile('./files/given_network/network_configuration_line_data_no_shunt.csv')
bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack1.csv')
# bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack2.csv')
# bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack1_no_reactive_load.csv')

Sbase = 100 # MVA
Ubase = 230 # kV
max_iterations = 30
tolerance = 1e-6


# Newton Raphson Load Flow Algorithm

In [2]:
from NRLF import NewtonRaphson

XR_ratio = None # None is default value
Q_lim = 0.65

NewtonRaphson(bus_data=bus_data, 
              line_data=line_data, 
              Sbase=Sbase,
              Ubase=Ubase, 
              max_iterations=max_iterations, 
              tolerance=tolerance, 
              Q_lim=Q_lim,
              XR_ratio=XR_ratio
              )


\begin{tabular}{lrlrrrr}
\toprule
 & Bus & Type & Voltage [pu] & Angle [deg] & P [pu] & Q [pu] \\
\midrule
0 & 1 & Slack & 243.800000 & 0.000000 & 131.122000 & 90.816000 \\
1 & 2 & PV & 230.000000 & -2.061000 & 20.000000 & -71.593000 \\
2 & 3 & PQ & 227.067000 & -4.637000 & -45.000000 & -15.000000 \\
3 & 4 & PQ & 226.350000 & -4.957000 & -40.000000 & -5.000000 \\
4 & 5 & PQ & 223.490000 & -5.765000 & -60.000000 & -10.000000 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrlrrrr}
\toprule
 & Bus & Type & Voltage [pu] & Angle [deg] & P [pu] & Q [pu] \\
\midrule
0 & 1 & Slack & 1.060000 & 0.000000 & 1.311000 & 0.908000 \\
1 & 2 & PV & 1.000000 & -2.061000 & 0.200000 & -0.716000 \\
2 & 3 & PQ & 0.987000 & -4.637000 & -0.450000 & -0.150000 \\
3 & 4 & PQ & 0.984000 & -4.957000 & -0.400000 & -0.050000 \\
4 & 5 & PQ & 0.972000 & -5.765000 & -0.600000 & -0.100000 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrrrrrrrr}
\toprule
 & From bus & To bus & P ij[MW] & Q ij[MVAr] & P ji[MW] & Q ji[M

# Decoupled Load Flow

In [3]:
from DLF import DLF

Q_lim = 0.65

DLF(bus_data=bus_data, 
    line_data=line_data, 
    Sbase=Sbase,
    Ubase=Ubase, 
    max_iterations=max_iterations, 
    tolerance=tolerance, 
    Q_lim=Q_lim
    )


\begin{tabular}{lrlrrrr}
\toprule
 & Bus & Type & Voltage [pu] & Angle [deg] & P [pu] & Q [pu] \\
\midrule
0 & 1 & Slack & 243.800000 & 0.000000 & 131.122000 & 90.816000 \\
1 & 2 & PV & 230.000000 & -2.061000 & 20.000000 & -71.593000 \\
2 & 3 & PQ & 227.067000 & -4.637000 & -45.000000 & -15.000000 \\
3 & 4 & PQ & 226.350000 & -4.957000 & -40.000000 & -5.000000 \\
4 & 5 & PQ & 223.490000 & -5.765000 & -60.000000 & -10.000000 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrlrrrr}
\toprule
 & Bus & Type & Voltage [pu] & Angle [deg] & P [pu] & Q [pu] \\
\midrule
0 & 1 & Slack & 1.060000 & 0.000000 & 1.311000 & 0.908000 \\
1 & 2 & PV & 1.000000 & -2.061000 & 0.200000 & -0.716000 \\
2 & 3 & PQ & 0.987000 & -4.637000 & -0.450000 & -0.150000 \\
3 & 4 & PQ & 0.984000 & -4.957000 & -0.400000 & -0.050000 \\
4 & 5 & PQ & 0.972000 & -5.765000 & -0.600000 & -0.100000 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrrrrrrrr}
\toprule
 & From bus & To bus & P ij[MW] & Q ij[MVAr] & P ji[MW] & Q ji[M

# Fast Decoupled Load Flow

In [4]:
from FDLF import FDLF

# True for calculation partially through iteration or False for calculation at end of iteration.
# If False, increase Q_lim. There is a fault that collide with typeSwitching in FDLF.
iterate_partial_orEnd = False 
Q_lim = 0.65
XR_ratio = None # None is Default value

FDLF(bus_data=bus_data, 
    line_data=line_data, 
    Sbase=Sbase,
    Ubase=Ubase, 
    max_iterations=max_iterations, 
    tolerance=tolerance,
    Q_lim=Q_lim,
    iterate_partial_orEnd=iterate_partial_orEnd,
    XR_ratio=XR_ratio
    )

\begin{tabular}{lrlrrrr}
\toprule
 & Bus & Type & Voltage [pu] & Angle [deg] & P [pu] & Q [pu] \\
\midrule
0 & 1 & Slack & 243.800000 & 0.000000 & 131.122000 & 90.816000 \\
1 & 2 & PV & 230.000000 & -2.061000 & 20.000000 & -71.593000 \\
2 & 3 & PQ & 227.067000 & -4.637000 & -45.000000 & -15.000000 \\
3 & 4 & PQ & 226.350000 & -4.957000 & -40.000000 & -5.000000 \\
4 & 5 & PQ & 223.490000 & -5.765000 & -60.000000 & -10.000000 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrlrrrr}
\toprule
 & Bus & Type & Voltage [pu] & Angle [deg] & P [pu] & Q [pu] \\
\midrule
0 & 1 & Slack & 1.060000 & 0.000000 & 1.311000 & 0.908000 \\
1 & 2 & PV & 1.000000 & -2.061000 & 0.200000 & -0.716000 \\
2 & 3 & PQ & 0.987000 & -4.637000 & -0.450000 & -0.150000 \\
3 & 4 & PQ & 0.984000 & -4.957000 & -0.400000 & -0.050000 \\
4 & 5 & PQ & 0.972000 & -5.765000 & -0.600000 & -0.100000 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrrrrrrrr}
\toprule
 & From bus & To bus & P ij[MW] & Q ij[MVAr] & P ji[MW] & Q ji[M

# DC Power Flow

In [5]:
from DCPF import DCPF

DCPF(bus_data=bus_data,
     line_data=line_data,
     Sbase=Sbase
     )


 {'Dirac_2': -0.05802913144535791, 'Dirac_3': -0.10715935223736155, 'Dirac_4': -0.11354618258659717, 'Dirac_5': -0.1303315495575311} 

\begin{tabular}{lrrrrr}
\toprule
 & 0 & 1 & 2 & 3 & 4 \\
\midrule
0 & 0.000000 & 87.043697 & 40.184757 & 0.000000 & 0.000000 \\
1 & -87.043697 & 0.000000 & 24.565110 & 27.758526 & 54.226814 \\
2 & -40.184757 & -24.565110 & 0.000000 & 19.160491 & 0.000000 \\
3 & 0.000000 & -27.758526 & -19.160491 & 0.000000 & 6.294513 \\
4 & 0.000000 & -54.226814 & 0.000000 & -6.294513 & 0.000000 \\
\bottomrule
\end{tabular}

--- 0.0030050277709960938 seconds ---
